In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
pt_london_2025_invited = "https://fabtcg.com/en/organised-play/2025/pro-tour-london/pro-tour-london-invited-players/"
pt_london_2025 = "https://fabtcg.com/en/coverage/pro-tour-london/decklist/{}/"

In [62]:
def get_player_list(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    player_list_table = soup.find("table")

    table_row = player_list_table.find_all("tr")

    return [data.text.strip() for data in (row_data[1] for row_data in (row.find_all("td") for row in table_row[1:]))]

def get_decklist(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    decklist = soup.find_all("td")

    # Hero = decklist[4]
    

    return decklist


In [18]:
player_list = get_player_list(pt_london_2025_invited)

In [63]:
tbl = get_decklist("https://fabtcg.com/en/coverage/pro-tour-london/decklist/57895736/")
tbl

[<td>Aaron Moore (57895736)</td>,
 <td>Apr. 11, 2025</td>,
 <td>
 <a href="/en/coverage/pro-tour-london/">
             Pro Tour: London
             </a>
 </td>,
 <td>Classic Constructed</td>,
 <td>Fai, Rising Rebellion</td>,
 <td><div class="hover_img">
                             1 x <a href="https://d2wlb52bya4y8z.cloudfront.net/media/cards/normal/HNT143-RF.webp" onclick="return false;" target="_blank">Dragonscaler Flight Path
                             <span><img loading="lazy" src="https://d2wlb52bya4y8z.cloudfront.net/media/cards/large/HNT143-RF.webp"/></span></a>
 </div></td>,
 <td><div class="hover_img">
                             1 x <a href="https://d2wlb52bya4y8z.cloudfront.net/media/cards/normal/UPR084-RF.webp" onclick="return false;" target="_blank">Flamescale Furnace
                             <span><img loading="lazy" src="https://d2wlb52bya4y8z.cloudfront.net/media/cards/large/UPR084-RF.webp"/></span></a>
 </div></td>,
 <td><div class="hover_img">
              

<td>Enigma, Ledger of Ancestry</td>